In [1]:
import os
import tempfile
import matplotlib.pyplot as plt
import glob
from pymongo import MongoClient, InsertOne
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

database = "vnw_job"
col = "items" 

2023-05-11 12:29:54.022077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class QueryModel(tf.keras.Model):
    """Model for encoding user queries."""

    def __init__(self, layer_sizes):
        """Model for encoding user queries.

        Args:
          layer_sizes:
            A list of integers where the i-th entry represents the number of units
            the i-th layer contains.
        """
        super().__init__()

        # We first use the user model for generating embeddings.
        self.embedding_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None, name='user_stringlookup_layer'),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, 64, name='user_embedding_layer'),
        ], name='user_embedding_model')

    #     # Then construct the layers.
    #     self.dense_layers = tf.keras.Sequential()

    #     # Use the ReLU activation for all but the last layer.
    #     for layer_size in layer_sizes[:-1]:
    #       self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    #     # No activation for the last layer.
    #     for layer_size in layer_sizes[-1:]:
    #       self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        return self.embedding_model(inputs["user_id"])


In [3]:
@tf.keras.utils.register_keras_serializable()
def tokenization(t):
    return tf.strings.split(t, ',')

In [5]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.job_id_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_job_ids,mask_token=None),
      tf.keras.layers.Embedding(len(unique_job_ids) + 1, 32)
    ])
    self.location_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=100,
          standardize=None,
          split=tokenization,
          vocabulary=unique_locations,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_locations) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    self.level_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=100,
          standardize=None,
          split=tokenization,
          vocabulary=unique_levels,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_levels) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    self.category_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=100,
          standardize=None,
          split=tokenization,
          vocabulary=unique_category,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_category) + 1, 64),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    self.skill_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=11000,
          standardize=None,
          split=tokenization,
          vocabulary=unique_skills,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_skills) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    # self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = tf.concat([
        self.job_id_embedding(inputs["job_id"]),
        self.category_embedding(inputs["category"]),
        self.location_embedding(inputs["location"]),
        self.level_embedding(inputs["level"]),
        self.skill_embedding(inputs["skill_text"])
    ], axis=1)
    return self.dense_layers(feature_embedding)

In [8]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.query_model: tf.keras.Model = user_model
    self.candidate_model: tf.keras.Model = movie_model
    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"]
    })
    movie_embeddings = self.candidate_model({
        "job_id": features["job_id"],
        "job_title": features["job_title"],
        "category": features["category"],
        "location": features["location"],
        "skill_text": features["skill_text"],
        "level": features["level"],
    })

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)


In [9]:
'''Load data vs metadata'''
mongo = MongoClient()
df_meta = pd.DataFrame(list(mongo["data_8"][col].find()))
df_meta['jobTitle'] = df_meta.categoricalProps.apply(lambda x: x["jobTitle"][0])
df_meta['skills'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["skills"]))
df_meta['industries'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["industries"]))
df_meta['locations'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["locations"]))
df_meta['jobLevel'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["jobLevel"]))
df_meta['availableDate'] = df_meta.dateProps.apply(lambda x: x["availableDate"])
df = pd.read_parquet(glob.glob("/home/spark/ylv/data/navidata/*.parquet"))
'''Load events data from Mongo'''
big_data_lst = ["1633404", "1627161", "1627748", "1629553", "1639460", "1627886", "1634491", "1618860", "1621708", "1622680",
                "1630360", "1635701", "1627675", "1639375", "1607880", "1620068", "1620566", "1620569", "1620743",
                "1623842", "1628767", "1634087", "1634973", "1639216", "1615953"]

dba_lst = ["1611833", "1622983", "1628567", "1634484", '1635393', '1637868', "1639233", "1612768", "1619047", '1619206',
           "1620547", '1622265', "1622344", "1623175", "1623753", "1637169"]
# peer_users = [i['entityId'] for i in list(mongo['data_8']['data'].find({"targetEntityId": "1631782"}, {'_id': 0}))]
# df = pd.DataFrame(list(mongo['data_8']['data'].find({'entityId': {"$in": peer_users}})) + [{'entityId': 'user_test_001', 'targetEntityId': '1631782'}])
df1 = pd.DataFrame(list(mongo['data_8']['events_1'].find({}, {"_id": 0})) + [
    {"entityId": "user_test_002", "targetEntityId": "1631782"}] + [{"entityId": f"user_autoadd_{np.random.choice(range(200))}", "targetEntityId": np.random.choice(big_data_lst)} for _ in range(20000)
                                                                   ]
                  + [
                      {"entityId": f"user_autoadd_{np.random.choice(range(200, 300))}",
                       "targetEntityId": np.random.choice(dba_lst)} for _ in range(10000)
                  ])

df = df.append(df1)
ratings = df.merge(df_meta, left_on="targetEntityId", right_on="_id", how="left")
ratings = ratings.dropna()
# rating_1st, rating_2nd = train_test_split(ratings, test_size=0.05)

/tmp/ipykernel_2509816/2209504810.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


In [10]:
movies = tf.data.Dataset.from_tensor_slices({
        "job_id": df_meta._id.values.tolist(),
        "category": df_meta.industries.values.tolist(),
        "location": df_meta.locations.values.tolist(),
        "level": df_meta.jobLevel.values.tolist(),
        "job_title": df_meta.jobTitle.values.tolist(),
        "skill_text": df_meta.skills.values.tolist()
    })

2023-05-11 12:31:15.265157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 12:31:15.290833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 12:31:15.291001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 12:31:15.291391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [11]:
tensor_slices = {
        "user_id": ratings.entityId.values.tolist(),
        "job_id": ratings.targetEntityId.values.tolist(),
        "location": ratings.locations.values.tolist(),
        "category": ratings.industries.values.tolist(),
        "level": ratings.jobLevel.values.tolist(),
        "job_title": ratings.jobTitle.values.tolist(),
        "skill_text": ratings.skills.values.tolist()
    }

In [12]:
%%time
ds_rating_1st = tf.data.Dataset.from_tensor_slices(tensor_slices)

CPU times: user 54.2 s, sys: 1.86 s, total: 56.1 s
Wall time: 56.1 s


In [13]:
%%time

unique_job_titles = np.unique(df_meta.jobTitle.values.tolist())
unique_job_ids = np.unique(df_meta._id.values.tolist())
unique_user_ids = np.unique(ratings.entityId.values.tolist())
unique_locations = np.unique(reduce(lambda x, y: x + "," + y, df_meta.locations.values.tolist()).split(","))
unique_skills = np.unique(reduce(lambda x, y: x + "," + y, df_meta.skills.values.tolist()).split(","))[1:]
# job_title_vocabs = np.unique(reduce(lambda x, y: x + y, df_meta.title.str.split(" ").tolist()))
unique_category = np.unique(reduce(lambda x, y: x + "," + y, df_meta.industries.values.tolist()).split(","))
unique_levels = np.unique(reduce(lambda x, y: x + "," + y, df_meta.jobLevel.values.tolist()).split(","))

CPU times: user 6.38 s, sys: 1.08 s, total: 7.46 s
Wall time: 7.46 s


In [14]:
'''Train'''
cached_train_1st = ds_rating_1st.shuffle(buffer_size=500).batch(4096)
num_epochs = 50

user_model = QueryModel([128, 64, 32])
item_model = CandidateModel([256, 128, 64])

In [15]:
model = MovielensModel(user_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
history = model.fit(
    cached_train_1st,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

2023-05-11 12:32:50.667573: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f16401718a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-11 12:32:50.667591: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-05-11 12:32:50.669946: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-11 12:32:50.710010: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:50.735031: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled clust

2023-05-11 12:32:52.341845: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:52.415336: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:52.501441: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:32:54.179093: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:54.250094: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:54.320835: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:32:55.966534: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:56.046434: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:56.118679: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:32:57.822196: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:57.899626: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:32:57.972509: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

...

2023-05-11 12:53:16.255735: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:53:22.446466: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:53:25.456545: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:54:25.261422: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:54:30.236018: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:54:30.875046: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:55:24.566165: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:55:24.645024: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:55:28.490918: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:56:33.477872: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:56:34.423924: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:56:34.689852: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 12:57:35.555561: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:57:35.638003: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 12:57:39.527501: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

In [16]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model, k=50)
index.index_from_dataset(
        movies.batch(500).map(lambda x: (x["job_id"], model.candidate_model(x))))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [55]:
_, preds = index({"user_id": np.array(["user_test_040"])})
idx = list(map(lambda x: x.decode('ascii'), preds.numpy()[0]))
df_check = df_meta[df_meta._id.isin(idx)][["_id", "jobTitle", "industries", "jobLevel", "locations", "skills"]]
df_check["no"] = df_check._id.apply(lambda x: idx.index(x))
df_check.sort_values(by=["no"])

,_id,jobTitle,industries,jobLevel,locations,skills,no
2018,1622344,senior database administrator,IT - Software,Experienced (non-manager),Ho Chi Minh,"data warehouse,database design,elastic search,...",0
3749,1624532,platform qa specialist,"Retail/Wholesale,E-commerce",Experienced (non-manager),Ho Chi Minh,"qa,policy development,internal control,complia...",1
6984,1628625,chuyen vien nghien cuu cong nghe va giai phap ...,"IT - Software,IT - Hardware/Networking,Fintech",Experienced (non-manager),Ha Noi,"tieng anh,java,webservice,oracle,sql server",2
4114,1625010,junior tester qa qc executive,"IT - Software,E-commerce",Experienced (non-manager),Ha Noi,"app tester,website tester",3
7170,1628815,voice over lte 5g software engineer,"IT - Software,Telecommunications,IT - Hardware...",Experienced (non-manager),"Ha Noi,Ho Chi Minh","mobile technology,ip pbx,ims,oracle,asterisk",4
1015,1619196,mobile developer,"IT - Software,Fintech",Experienced (non-manager),"Ha Noi,Ho Chi Minh","react native,java,swift,ios,android",5
7060,1628696,consultant salesforce developer hcm hn,IT - Software,Experienced (non-manager),"Ha Noi,Ho Chi Minh","salesforce,lap trinh vien software developer",6
6952,1628575,chuyen vien quan tri va phat trien ung dung ma...,"IT - Software,Fintech",Experienced (non-manager),Ha Noi,"java,.net,c,iso 8583,tieng anh",7
9445,1631614,hcm senior java backend java fullstack developer,IT - Software,Experienced (non-manager),Ho Chi Minh,"java,spring web mvc",8
818,1616842,software qa automation engineer up to 2 000,"IT - Software,IT - Hardware/Networking,QA/QC",Experienced (non-manager),Ho Chi Minh,"testers,qa,test case,software qa,computer science",9


In [24]:
# model.query_model.save('/home/spark/ylv/workplace/rec_online_training/navi_user_model_5')
# model.candidate_model.save('/home/spark/ylv/workplace/rec_online_training/navi_item_model_5')

INFO:tensorflow:Assets written to: /home/spark/ylv/workplace/rec_online_training/navi_user_model_5/assets
INFO:tensorflow:Assets written to: /home/spark/ylv/workplace/rec_online_training/navi_item_model_5/assets


In [91]:
user_model_path = "/home/spark/ylv/workplace/rec_online_training/navi_user_model_5"
item_model_path = "/home/spark/ylv/workplace/rec_online_training/navi_item_model_5"

In [92]:
def update_item_recs(user, recs, score):
    return UpdateOne({"_id": user}, {
        "$set": {
            "items": dict(zip(list(map(lambda x: x.decode("ascii"), recs)), scale_10(score)))
        }
    }, upsert=True)

In [93]:
class QueryModelUpdated(tf.keras.Model):
    """Model for encoding user queries."""

    def __init__(self, vocab_new, updated_embedding):
        """Model for encoding user queries.

        Args:
          layer_sizes:
            A list of integers where the i-th entry represents the number of units
            the i-th layer contains.
        """
        super().__init__()

        # We first use the user model for generating embeddings.
        self.user_embedding_layer_new = tf.keras.layers.Embedding(
            len(vocab_new), 64, name="new_user_embedding"
        )
        self.user_embedding_layer_new.build(input_shape=[None])
        self.user_embedding_layer_new.embeddings.assign(updated_embedding)
        self.embedding_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=vocab_new, mask_token=None, name='user_stringlookup_layer_1'),
            self.user_embedding_layer_new
        ], name='user_embedding_model_1')

    #     # Then construct the layers.
    #     self.dense_layers = tf.keras.Sequential()

    #     # Use the ReLU activation for all but the last layer.
    #     for layer_size in layer_sizes[:-1]:
    #       self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    #     # No activation for the last layer.
    #     for layer_size in layer_sizes[-1:]:
    #       self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        return self.embedding_model(inputs["user_id"])

In [94]:
class MovielensModel(tfrs.Model):

    def __init__(self, user_model, movie_model):
        super().__init__()
        self.query_model: tf.keras.Model = user_model
        self.candidate_model: tf.keras.Model = movie_model
        self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.candidate_model),
            ),
        )

    def compute_loss(self, features, training=False):
        # We only pass the user id and timestamp features into the query model. This
        # is to ensure that the training inputs would have the same keys as the
        # query inputs. Otherwise the discrepancy in input structure would cause an
        # error when loading the query model after saving it.
        query_embeddings = self.query_model({
            "user_id": features["user_id"]
        })
        movie_embeddings = self.candidate_model({
            "job_id": features["job_id"],
            "job_title": features["job_title"],
            "category": features["category"],
            "location": features["location"],
            "skill_text": features["skill_text"],
            "level": features["level"],
        })

        return self.task(
            query_embeddings, movie_embeddings, compute_metrics=not training)

In [95]:
'''Load data vs metadata'''
mongo = MongoClient()

'''Load metadata from Mongo'''
df_meta = pd.DataFrame(list(mongo['data_8']['items'].find()))
df_meta['jobTitle'] = df_meta.categoricalProps.apply(lambda x: x["jobTitle"][0])
df_meta['skills'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["skills"]))
df_meta['industries'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["industries"]))
df_meta['locations'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["locations"]))
df_meta['jobLevel'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["jobLevel"]))
df_meta['availableDate'] = df_meta.dateProps.apply(lambda x: x["availableDate"])
# df = pd.read_parquet(glob.glob("/home/spark/ylv/data/navidata/*.parquet"))

'''Load events data from Mongo'''
big_data_lst = ["1633404", "1627161", "1627748", "1629553", "1639460", "1627886", "1634491", "1618860", "1621708", "1622680",
                "1630360", "1635701", "1627675", "1639375", "1607880", "1620068", "1620566", "1620569", "1620743",
                "1623842", "1628767", "1634087", "1634973", "1639216", "1615953"]

dba_lst = ["1611833", "1622983", "1628567", "1634484", '1635393', '1637868', "1639233", "1612768", "1619047", '1619206',
           "1620547", '1622265', "1622344", "1623175", "1623753", "1637169"]
# peer_users = [i['entityId'] for i in list(mongo['data_8']['data'].find({"targetEntityId": "1631782"}, {'_id': 0}))]
# df = pd.DataFrame(list(mongo['data_8']['data'].find({'entityId': {"$in": peer_users}})) + [{'entityId': 'user_test_001', 'targetEntityId': '1631782'}])
online_df = pd.DataFrame(list(mongo['data_8']['events_1'].find({}, {"_id": 0})) + [
    {"entityId": "user_test_044", "targetEntityId": "1622344"}] + [{"entityId": f"user_autoadd_{np.random.choice(range(500))}", "targetEntityId": np.random.choice(big_data_lst)} for _ in range(50000)
                                                                   ]
                  + [
                      {"entityId": f"user_autoadd_{np.random.choice(range(500, 1000))}",
                       "targetEntityId": np.random.choice(dba_lst)} for _ in range(30000)
                  ])

online_ratings = online_df.merge(df_meta, left_on="targetEntityId", right_on="_id", how="left")
online_ratings = online_ratings.dropna()

In [96]:
movies = tf.data.Dataset.from_tensor_slices({
        "job_id": df_meta._id.values.tolist(),
        "category": df_meta.industries.values.tolist(),
        "location": df_meta.locations.values.tolist(),
        "level": df_meta.jobLevel.values.tolist(),
        "job_title": df_meta.jobTitle.values.tolist(),
        "skill_text": df_meta.skills.values.tolist()
    })

In [97]:
user_model = tf.keras.models.load_model(user_model_path)
item_model = tf.keras.models.load_model(item_model_path)
vocab_base = user_model.get_layer("user_embedding_model").get_layer("user_stringlookup_layer").get_vocabulary()
embedding_weights_base = \
    user_model.get_layer("user_embedding_model").get_layer("user_embedding_layer").get_weights()[0]
new_users = list(set(online_ratings.entityId.unique()) - set(vocab_base))
print(new_users)
if len(new_users) > 0:
    vocab_new = vocab_base + new_users
    updated_embedding = tf.keras.utils.warmstart_embedding_matrix(
        base_vocabulary=vocab_base,
        new_vocabulary=vocab_new,
        base_embeddings=embedding_weights_base,
        new_embeddings_initializer="uniform",
    )
    updated_embedding_variable = tf.Variable(updated_embedding)
    user_model1 = QueryModelUpdated(vocab_new, updated_embedding)

['user_autoadd_840', 'user_autoadd_379', 'user_autoadd_352', 'user_autoadd_764', 'user_autoadd_338', 'user_autoadd_305', 'user_autoadd_432', 'user_autoadd_457', 'user_autoadd_365', 'user_autoadd_957', 'user_autoadd_870', 'user_autoadd_530', 'user_autoadd_546', 'user_autoadd_995', 'user_autoadd_433', 'user_autoadd_888', 'user_autoadd_751', 'user_autoadd_637', 'user_autoadd_885', 'user_autoadd_438', 'user_autoadd_927', 'user_autoadd_621', 'user_autoadd_889', 'user_autoadd_548', 'user_autoadd_812', 'user_autoadd_985', 'user_autoadd_958', 'user_autoadd_828', 'user_autoadd_947', 'user_autoadd_477', 'user_autoadd_783', 'user_autoadd_521', 'user_autoadd_660', 'user_autoadd_851', 'user_autoadd_977', 'user_autoadd_821', 'user_autoadd_676', 'user_autoadd_452', 'user_autoadd_879', 'user_autoadd_898', 'user_autoadd_701', 'user_autoadd_595', 'user_autoadd_973', 'user_autoadd_431', 'user_autoadd_972', 'user_autoadd_580', 'user_autoadd_376', 'user_autoadd_631', 'user_autoadd_776', 'user_autoadd_695',

In [98]:
tensor_slices_2nd = {
        "user_id": online_ratings.entityId.values.tolist(),
        "job_id": online_ratings.targetEntityId.values.tolist(),
        "location": online_ratings.locations.values.tolist(),
        "category": online_ratings.industries.values.tolist(),
        "level": online_ratings.jobLevel.values.tolist(),
        "job_title": online_ratings.jobTitle.values.tolist(),
        "skill_text": online_ratings.skills.values.tolist()
    }

In [99]:
ds_rating_2nd = tf.data.Dataset.from_tensor_slices(tensor_slices_2nd)
cached_train_2nd = ds_rating_2nd.shuffle(buffer_size=500).batch(4096)

In [100]:
num_epochs = 50
model1 = MovielensModel(user_model1, item_model)
model1.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
history1 = model1.fit(
    cached_train_2nd,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

2023-05-11 14:10:00.276474: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:00.709011: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:00.916460: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 14:10:04.972207: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:05.187402: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:05.311219: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 14:10:10.420458: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:10.792086: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:11.041510: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 14:10:19.269810: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:20.888390: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:21.286473: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 14:10:34.676276: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:35.847185: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:36.012304: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 14:10:51.674774: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:53.786053: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:10:54.018181: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-05-11 14:11:45.816510: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:11:47.219025: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-05-11 14:11:54.405696: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

In [101]:
index1 = tfrs.layers.factorized_top_k.BruteForce(model1.query_model, k=50)
index1.index_from_dataset(
        movies.batch(500).map(lambda x: (x["job_id"], model1.candidate_model(x))))

In [102]:
_, preds = index1({"user_id": np.array(["user_test_046"])})
idx = list(map(lambda x: x.decode('ascii'), preds.numpy()[0]))
df_check = df_meta[df_meta._id.isin(idx)][["_id", "jobTitle", "industries", "jobLevel", "locations", "skills"]]
df_check["no"] = df_check._id.apply(lambda x: idx.index(x))
df_check.sort_values(by=["no"])

,_id,jobTitle,industries,jobLevel,locations,skills,no
9303,1631467,qa game tester,"Entry level,IT - Software,QA/QC",Experienced (non-manager),Ho Chi Minh,"game testing,quality assurance,game tester,man...",0
11149,1633783,khoi cong nghe thong tin chuyen vien quan tri ...,"IT - Software,Banking",Experienced (non-manager),Ho Chi Minh,sql oracle,1
8638,1630624,hcm nhan vien cskh line tieng nhat,Customer Service,Fresher/Entry level,Ho Chi Minh,"tieng nhat n3,tieng nhat n2",2
1975,1622265,senior database administrator,IT - Software,Experienced (non-manager),Ho Chi Minh,"data warehouse,database design,elastic search,...",3
2018,1622344,senior database administrator,IT - Software,Experienced (non-manager),Ho Chi Minh,"data warehouse,database design,elastic search,...",4
8243,1630160,ho data quality specialist,"Banking,Fintech",Experienced (non-manager),Ho Chi Minh,"data quality,data,sql oracle,oracle,sql sever",5
12373,1635354,ui designer chinese or english,"Advertising/Promotion/PR,Arts/Design,Internet/...",Experienced (non-manager),Ho Chi Minh,"ui design,p oshop,illustrator,graphic design,f...",6
15413,1639007,chuyen vien cong nghe thong tin,"IT - Software,Airlines/Tourism,IT - Hardware/N...",Experienced (non-manager),Ho Chi Minh,"lap trinh web,.net,lap trinh di dong,javascrip...",7
11703,1634484,ho khoi cntt chuyen vien ban van hanh he thong...,"IT - Software,Banking,IT - Hardware/Networking",Experienced (non-manager),Ha Noi,"oracle,oracle goldengate,my sql,sql server,linux",8
3622,1624417,hcm city devops technical architect global tec...,IT - Software,Experienced (non-manager),Ho Chi Minh,"software system architect,devops",9


In [104]:
from pymongo import MongoClient, UpdateOne

def scale_10(arr):
    arr = (arr - min(arr) + 1)/(max(arr) - min(arr))
    return list(map(float, arr*10/max(arr)))


def update_item_recs(user, recs, score):
    return UpdateOne({"_id": user}, {
        "$set": {
            "items": dict(zip(list(map(lambda x: x.decode("ascii"), recs)), scale_10(score)))
        }
    }, upsert=True)

updated_users = online_ratings.entityId.unique().tolist()
batch_predict = 40000
for i in range(len(updated_users) // batch_predict + 1):
    predict_users = updated_users[i * batch_predict:(i + 1) * batch_predict]
    score, preds = index1({"user_id": np.array(predict_users)})
    mongo['data_8'].User_Recs.bulk_write(
        list(map(lambda u, r, s: update_item_recs(u, r, s), predict_users, preds.numpy(), score.numpy()))
    )